<h1><center>The Battle of Neighborhoods</center></h1>

This **capstone project** has been submitted as part of the requirements for completion of the **IBM Data Science Professional Certificate** on Coursera. In general, this project would be encompassing a series of Data Science techniques, including, but not limited to, Web Scraping (using BeautifulSoup and Requests), Data Cleaning, Data Wrangling and Machine Learning (K-Means clustering algorithm).

### Importing required libraries and packages

In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


### Scraping web for neighborhood data of Berlin (WikiPedia)

In [11]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Localities_of_Berlin").text
soup = BeautifulSoup(data, 'lxml')
textList = []
neighborhoodList = []

In [17]:
# append the data into the list
neighborhoodList.clear()

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df1 = df.iloc[1:]
berlin_df = df1.reset_index(drop=True)
berlin_df.head()

,Neighborhood
0,Adlershof
1,Alt-Hohenschönhausen
2,Alt-Treptow
3,Altglienicke
4,Baumschulenweg


In [13]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in berlin_df["Neighborhood"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
berlin_df['Latitude'] = df_coords['Latitude']
berlin_df['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(berlin_df.shape)
berlin_df.head()

(96, 3)


,Neighborhood,Latitude,Longitude
0,Adlershof,52.43779,13.54778
1,Alt-Hohenschönhausen,52.54706,13.50055
2,Alt-Treptow,52.49350,13.45711
3,Altglienicke,52.42006,13.53969
4,Baumschulenweg,52.46669,13.48840


In [14]:
# save the DataFrame as CSV file
berlin_df.to_csv("berlin_neighborhoods.csv", index=False)

#### Visualise map with neighborhoods superimposed

In [15]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin is 52.5170365, 13.3888599.


In [16]:

# create map of Toronto using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

#### Using Foursquare API